In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import keras
import nltk
import spacy

Using TensorFlow backend.


In [3]:
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from gensim import logging
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as  pd
import seaborn as sns
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import json
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import MWETokenizer

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
word_files = ['located_relation.txt']
word_list = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip() for x in lines] 
    
    word_list.extend(word_list2)
word_list[0:5]

['Staying', 'staying', 'Stay', 'stay at', 'Visiting']

In [7]:
#remove duplicates
def ordered_set(in_list):
    out_list = []
    added = set()
    for val in in_list:
        if not val in added:
            out_list.append(val)
            added.add(val)
    return out_list

In [8]:
len(word_list)

63

In [9]:
word_list = ordered_set(word_list)

In [10]:
len(word_list)

63

In [11]:
mw_list = []
for x in word_list:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
mw_list[0:5]

[('stay', 'at'),
 ('Staying', 'at'),
 ('Located', 'in'),
 ('located', 'in'),
 ('close', 'to')]

In [12]:
for i in mw_list:
    if i == ('close','to'):
        print('found')

found


In [21]:
mwe = MWETokenizer(mw_list,separator=' ')

In [13]:
reviews = pd.read_csv("./location.csv")

In [14]:
reviews.shape

(13019, 1)

In [15]:
reviews.head()

,location
0,Staying at @ Home Executive Apartment is a goo...
1,Staying at @ Oasis Resort is a good choice whe...
2,Staying at @ T Boutique Hotel is a good choice...
3,Staying at AC Resort is a good choice when you...
4,Staying at Acqua Condominium is a good choice ...


In [16]:
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)

In [17]:
def spans(txt):
    tokens=mwe.tokenize(word_tokenize(txt))
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [18]:
for i in range(5):
    print("Review #",i+1)
    print(reviews.location[i])
    print()

Review # 1
Staying at @ Home Executive Apartment is a good choice when you are visiting Central Pattaya.

Review # 2
Staying at @ Oasis Resort is a good choice when you are visiting Wang Pong.

Review # 3
Staying at @ T Boutique Hotel is a good choice when you are visiting Khlong Wan.

Review # 4
Staying at AC Resort is a good choice when you are visiting Sairee beach.

Review # 5
Staying at Acqua Condominium is a good choice when you are visiting Jomtien Beach.



In [22]:
for i in range(10):
    print("Review #",i+1)
    print(reviews.location[i])
    for token in spans(reviews.location[i]):
        print(token)
        assert token[0]==reviews.location[i][token[1]:token[2]]
    print()

Review # 1
Staying at @ Home Executive Apartment is a good choice when you are visiting Central Pattaya.
('Staying at', 0, 10)
('@', 11, 12)
('Home', 13, 17)
('Executive', 18, 27)
('Apartment', 28, 37)
('is', 38, 40)
('a', 41, 42)
('good', 43, 47)
('choice', 48, 54)
('when', 55, 59)
('you', 60, 63)
('are', 64, 67)
('visiting', 68, 76)
('Central', 77, 84)
('Pattaya', 85, 92)
('.', 92, 93)

Review # 2
Staying at @ Oasis Resort is a good choice when you are visiting Wang Pong.
('Staying at', 0, 10)
('@', 11, 12)
('Oasis', 13, 18)
('Resort', 19, 25)
('is', 26, 28)
('a', 29, 30)
('good', 31, 35)
('choice', 36, 42)
('when', 43, 47)
('you', 48, 51)
('are', 52, 55)
('visiting', 56, 64)
('Wang', 65, 69)
('Pong', 70, 74)
('.', 74, 75)

Review # 3
Staying at @ T Boutique Hotel is a good choice when you are visiting Khlong Wan.
('Staying at', 0, 10)
('@', 11, 12)
('T', 13, 14)
('Boutique', 15, 23)
('Hotel', 24, 29)
('is', 30, 32)
('a', 33, 34)
('good', 35, 39)
('choice', 40, 46)
('when', 47, 51)
(

In [23]:
for i in range(1000):
    word_ls = []
    #print("Review #",i+1)
    for token in spans(reviews.location[i]):
        #print(token)
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        my_tuple = token[1],token[2]
        #my_tuples = ' , '.join(map(str, my_tuple))
        if token[0] in word_list:
            word_ls.append(my_tuple)
            lis_lo = ' , '.join(map(str, word_ls))
    print(lis_lo)

(0, 10) , (68, 76)
(0, 10) , (56, 64)
(0, 10) , (60, 68)
(0, 10) , (51, 59)
(0, 10) , (59, 67)
(0, 10) , (58, 66)
(0, 10) , (65, 73)
(0, 10) , (75, 83)
(0, 10) , (72, 80)
(0, 10) , (85, 93)
(0, 10) , (56, 64)
(0, 10) , (61, 69)
(0, 10) , (63, 71)
(0, 10) , (66, 74)
(0, 10) , (58, 66)
(0, 10) , (64, 72)
(0, 10) , (64, 72)
(0, 10) , (67, 75)
(0, 10) , (63, 71)
(0, 10) , (67, 75)
(0, 10) , (61, 69)
(0, 10) , (60, 68)
(0, 10) , (67, 75)
(0, 10) , (48, 56)
(0, 10) , (57, 65)
(0, 10) , (66, 74)
(0, 10) , (60, 68)
(0, 10) , (56, 64)
(0, 10) , (61, 69)
(0, 10) , (58, 66)
(0, 10) , (58, 66)
(0, 10) , (62, 70)
(0, 10) , (64, 72)
(0, 10) , (67, 75)
(0, 10) , (60, 68)
(0, 10) , (55, 63)
(0, 10) , (54, 62)
(0, 10) , (61, 69)
(0, 10) , (57, 65)
(0, 10) , (53, 61)
(0, 10) , (56, 64)
(0, 10) , (64, 72)
(0, 10) , (67, 75)
(0, 10) , (65, 73)
(0, 10) , (58, 66)
(0, 10) , (67, 75)
(0, 10) , (74, 82)
(0, 10) , (59, 67)
(0, 10) , (55, 63)
(0, 10) , (79, 87)
(0, 10) , (56, 64)
(0, 10) , (63, 71)
(0, 10) , (5

(8, 15) , (54, 58) , (73, 81)
(8, 15) , (54, 58) , (73, 81)
(8, 15) , (47, 55) , (107, 119)
(8, 15) , (47, 55) , (107, 119)
(8, 15) , (111, 123)
(8, 15) , (120, 132)
(28, 43) , (90, 98) , (104, 108) , (221, 227)
(35, 50) , (109, 115)
(25, 40) , (85, 93) , (99, 103) , (216, 222)
(47, 59) , (69, 79) , (109, 116) , (130, 139) , (246, 250)
(36, 48) , (58, 68) , (116, 124) , (130, 134)
(40, 52) , (62, 72) , (152, 156)
(41, 53) , (63, 73) , (100, 107) , (121, 130) , (224, 228)
(39, 51) , (61, 71) , (107, 115) , (121, 125) , (211, 218) , (232, 241)
(45, 57) , (67, 77) , (102, 109) , (123, 132) , (227, 231)
(15, 30) , (89, 95)
(21, 36) , (83, 91) , (97, 101) , (187, 194) , (208, 217) , (280, 286)
(22, 37) , (101, 107)
(39, 51) , (61, 71) , (115, 123) , (129, 133) , (220, 227) , (241, 250)
(33, 45) , (55, 65) , (107, 115) , (121, 125)
(52, 64) , (74, 84) , (124, 132) , (138, 142) , (231, 238) , (252, 261)
(26, 41) , (73, 80) , (93, 102) , (165, 171)
(33, 48) , (95, 103) , (109, 113) , (198, 205

(43, 58) , (96, 103) , (117, 126) , (193, 199)
(28, 43) , (86, 93) , (107, 116) , (183, 189)
(49, 61) , (71, 81) , (172, 176)
(54, 66) , (76, 86) , (125, 133) , (139, 143)
(47, 59) , (69, 79) , (166, 170)
(18, 33) , (95, 101)
(47, 59) , (69, 79) , (116, 124) , (130, 134)
(44, 56) , (66, 76) , (110, 117) , (131, 140) , (241, 245)
(20, 35) , (98, 104)
(22, 37) , (83, 91) , (97, 101) , (214, 220)
(50, 62) , (72, 82) , (117, 125) , (131, 135)
(23, 38) , (80, 87) , (101, 110) , (177, 183)
(38, 50) , (60, 70) , (111, 119) , (125, 129)
(43, 55) , (65, 75) , (99, 106) , (120, 129) , (234, 238)
(32, 47) , (106, 112)
(27, 42) , (72, 79) , (93, 102) , (164, 170)
(19, 34) , (77, 85) , (91, 95) , (203, 209)
(40, 52) , (62, 72) , (111, 118) , (132, 141) , (238, 242)
(28, 40) , (50, 60) , (133, 137)
(40, 52) , (62, 72) , (107, 114) , (128, 137) , (234, 238)
(25, 40) , (81, 89) , (95, 99) , (208, 214)
(36, 48) , (58, 68) , (99, 106) , (120, 129) , (245, 249)
(41, 53) , (63, 73) , (110, 118) , (124, 12

In [46]:
for i in range(10):
    relation = []
    #print("Review #",i+1)
    for token in spans(reviews.location[i]):
        #print(token)
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        tuples = token[1],token[2] , "LocatedAt"
        if token[0] in word_list:
            relation.append(tuples)
            lis_lo = ' , '.join(map(str, relation))
            lis_lo3 = ((reviews.location[i]),({"entities": [(lis_lo)]}))
    print(lis_lo3)

('Staying at @ Home Executive Apartment is a good choice when you are visiting Central Pattaya.', {'entities': ["(0, 10, 'LocatedAt') , (68, 76, 'LocatedAt')"]})
('Staying at @ Oasis Resort is a good choice when you are visiting Wang Pong.', {'entities': ["(0, 10, 'LocatedAt') , (56, 64, 'LocatedAt')"]})
('Staying at @ T Boutique Hotel is a good choice when you are visiting Khlong Wan.', {'entities': ["(0, 10, 'LocatedAt') , (60, 68, 'LocatedAt')"]})
('Staying at AC Resort is a good choice when you are visiting Sairee beach.', {'entities': ["(0, 10, 'LocatedAt') , (51, 59, 'LocatedAt')"]})
('Staying at Acqua Condominium is a good choice when you are visiting Jomtien Beach.', {'entities': ["(0, 10, 'LocatedAt') , (59, 67, 'LocatedAt')"]})
('Staying at Aemjai Residence is a good choice when you are visiting Taling Ngam.', {'entities': ["(0, 10, 'LocatedAt') , (58, 66, 'LocatedAt')"]})
('Staying at Ai Phuket Hostel & Cafe is a good choice when you are visiting Phuket Town.', {'entities': 